# Read Data

In [1]:
import numpy as np
from scoring.make_rois_point import rois_point
from scoring.make_rois_image import rois_image
from scoring.make_rois_lesion import rois_lesion
from scoring.make_rois_speckle import rois_speckle
from scoring.make_rois_additional import rois_additional
from cubdl.das_torch import DAS_PW
from scoring.measure_picmus import measure_picmus
from scoring.measure_point import measure_point
from scoring.measure_image import measure_image
from scoring.measure_lesion import measure_lesion
from scoring.measure_speckle import measure_speckle
from scoring.measure_additional import measure_additional
import os
from datasets.PWDataLoaders import load_data
from cubdl.PlaneWaveData import PlaneWaveData
import h5py
from glob import glob
from scipy.signal import hilbert, convolve
from cubdl.PixelGrid import make_pixel_grid

In [2]:
class INSData(PlaneWaveData):
    """ Load data from INSERM. """
    def __init__(self, database_path, acq):
        # Make sure the selected dataset is valid
        moniker = "INS{:03d}".format(acq) + "*.hdf5"
        fname = [
            y for x in os.walk(database_path) for y in glob(os.path.join(x[0], moniker))
        ]
        assert fname, "File not found."

        # Load dataset
        f = h5py.File(fname[0], "r")

        # Phantom-specific parameters
        if acq == 1:
            sound_speed = 1521
        elif acq == 2:
            sound_speed = 1517
        elif acq == 3:
            sound_speed = 1506
        elif acq == 4:
            sound_speed = 1501
        elif acq == 5:
            sound_speed = 1506
        elif acq == 6:
            sound_speed = 1509
        elif acq == 7:
            sound_speed = 1490
        elif acq == 8:
            sound_speed = 1504
        elif acq == 9:
            sound_speed = 1473
        elif acq == 10:
            sound_speed = 1502
        elif acq == 11:
            sound_speed = 1511
        elif acq == 12:
            sound_speed = 1535
        elif acq == 13:
            sound_speed = 1453
        elif acq == 14:
            sound_speed = 1542
        elif acq == 15:
            sound_speed = 1539
        elif acq == 16:
            sound_speed = 1466
        elif acq == 17:
            sound_speed = 1462
        elif acq == 18:
            sound_speed = 1479
        elif acq == 19:
            sound_speed = 1469
        elif acq == 20:
            sound_speed = 1464
        elif acq == 21:
            sound_speed = 1508
        elif acq == 22:
            sound_speed = 1558
        elif acq == 23:
            sound_speed = 1463
        elif acq == 24:
            sound_speed = 1547
        elif acq == 25:
            sound_speed = 1477
        elif acq == 26:
            sound_speed = 1497
        else:
            sound_speed = 1540

        # Get data
        self.idata = np.array(f["channel_data"], dtype="float32")
        self.qdata = np.imag(hilbert(self.idata, axis=-1))
        self.angles = np.linspace(-16, 16, self.idata.shape[0]) * np.pi / 180
        self.fc = np.array(f["modulation_frequency"]).item()
        self.fs = np.array(f["sampling_frequency"]).item()
        self.c = sound_speed  # np.array(f["sound_speed"]).item()
        self.time_zero = -1 * np.array(f["start_time"], dtype="float32")[0]
        self.fdemod = 0
        self.ele_pos = np.array(f["element_positions"], dtype="float32").T
        self.ele_pos[:, 0] -= np.mean(self.ele_pos[:, 0])

        # For this dataset, time zero is the center point
        for i, a in enumerate(self.angles):
            self.time_zero[i] += self.ele_pos[-1, 0] * np.abs(np.sin(a)) / self.c

        # Validate that all information is properly included
        super().validate()

class MYOData(PlaneWaveData):
    """ Load data from Mayo Clinic. """

    def __init__(self, database_path, acq):
        # Make sure the selected dataset is valid
        moniker = "MYO{:03d}".format(acq) + "*.hdf5"
        fname = [
            y for x in os.walk(database_path) for y in glob(os.path.join(x[0], moniker))
        ]
        assert fname, "File not found."

        # Load dataset
        f = h5py.File(fname[0], "r")

        # Phantom-specific parameters
        if acq == 1:
            sound_speed = 1580
        elif acq == 2:
            sound_speed = 1583
        elif acq == 3:
            sound_speed = 1578
        elif acq == 4:
            sound_speed = 1572
        elif acq == 5:
            sound_speed = 1562
        else:
            sound_speed = 1581

        # Get data
        self.idata = np.array(f["channel_data"], dtype="float32")
        self.qdata = np.imag(hilbert(self.idata, axis=-1))
        self.angles = np.array(f["angles"])
        self.fc = np.array(f["modulation_frequency"]).item()
        self.fs = np.array(f["sampling_frequency"]).item()
        self.c = sound_speed  # np.array(f["sound_speed"]).item()
        self.time_zero = np.zeros((len(self.angles),), dtype="float32")
        self.fdemod = 0

        # Make the element positions based on L11-4v geometry
        pitch = 0.3e-3
        nelems = self.idata.shape[1]
        xpos = np.arange(nelems) * pitch
        xpos -= np.mean(xpos)
        self.ele_pos = np.stack([xpos, 0 * xpos, 0 * xpos], axis=1)

        # For this dataset, time zero is the center point
        for i, a in enumerate(self.angles):
            self.time_zero[i] = self.ele_pos[-1, 0] * np.abs(np.sin(a)) / self.c

        # Validate that all information is properly included
        super().validate()

class OSLData(PlaneWaveData):
    """ Load data from University of Oslo. """

    def __init__(self, database_path, acq):

        # Make sure the selected dataset is valid
        moniker = "OSL{:03d}".format(acq) + ".hdf5"
        fname = [
            y for x in os.walk(database_path) for y in glob(os.path.join(x[0], moniker))
        ]
        assert fname, "File not found."
        assert acq in [2, 3, 4, 5, 6, 7, 10], "Focused Data. Use FTDataLoaders"

        # Load dataset
        f = h5py.File(fname[0], "r")

        # Phantom-specific parameters
        if acq == 2:
            sound_speed = 1536
        elif acq == 3:
            sound_speed = 1543
        elif acq == 4:
            sound_speed = 1538
        elif acq == 5:
            sound_speed = 1539
        elif acq == 6:
            sound_speed = 1541
        elif acq == 7:
            sound_speed = 1540
        else:
            sound_speed = 1540

        # Get data
        self.idata = np.array(f["channel_data"], dtype="float32")
        self.qdata = np.imag(hilbert(self.idata, axis=-1))
        self.angles = np.array(f["transmit_direction"][0], dtype="float32")
        self.fc = np.array(f["modulation_frequency"]).item()
        self.fs = np.array(f["sampling_frequency"]).item()
        self.c = sound_speed  # np.array(f["sound_speed"]).item()
        self.time_zero = -1 * np.array(f["start_time"], dtype="float32")[0]
        self.fdemod = 0
        self.ele_pos = np.array(f["element_positions"], dtype="float32").T
        self.ele_pos[:, 0] -= np.mean(self.ele_pos[:, 0])

        # Validate that all information is properly included
        super().validate()

class UFLData(PlaneWaveData):
    """ Load data from UNIFI. """

    def __init__(self, database_path, acq):
        moniker = "UFL{:03d}".format(acq) + "*.hdf5"
        fname = [
            y for x in os.walk(database_path) for y in glob(os.path.join(x[0], moniker))
        ]
        assert fname, "File not found."

        # Load dataset
        f = h5py.File(fname[0], "r")

        # Phantom-specific parameters
        if acq == 1:
            sound_speed = 1526
        elif acq == 2 or acq == 4 or acq == 5:
            sound_speed = 1523
        else:
            sound_speed = 1525

        # Get data
        self.idata = np.array(f["channel_data"], dtype="float32")
        self.qdata = np.imag(hilbert(self.idata, axis=-1))
        self.angles = np.array(f["angles"]) * np.pi / 180
        self.fc = np.array(f["modulation_frequency"]).item()
        self.fs = np.array(f["channel_data_sampling_frequency"]).item()
        self.c = sound_speed  # np.array(f["sound_speed"]).item()
        self.time_zero = -1 * np.array(f["channel_data_t0"], dtype="float32")
        self.fdemod = self.fc

        # Make the element positions based on LA533 geometry
        pitch = 0.245e-3
        nelems = self.idata.shape[1]
        xpos = np.arange(nelems) * pitch
        xpos -= np.mean(xpos)
        self.ele_pos = np.stack([xpos, 0 * xpos, 0 * xpos], axis=1)

        # Make sure that time_zero is an array of size [nangles]
        if self.time_zero.size == 1:
            self.time_zero = np.ones_like(self.angles) * self.time_zero

        # Demodulate data and low-pass filter
        data = self.idata + 1j * self.qdata
        phase = np.reshape(np.arange(self.idata.shape[2], dtype="float"), (1, 1, -1))
        phase *= self.fdemod / self.fs
        data *= np.exp(-2j * np.pi * phase)
        dsfactor = int(np.floor(self.fs / self.fc))
        kernel = np.ones((1, 1, dsfactor), dtype="float") / dsfactor
        data = convolve(data, kernel, "same")
        data = data[:, :, ::dsfactor]
        self.fs /= dsfactor

        self.idata = np.real(data)
        self.qdata = np.imag(data)

        # Validate that all information is properly included
        super().validate()

class EUTData(PlaneWaveData):
    """ Load data from TU/e. """

    def __init__(self, database_path, acq):

        # Make sure the selected dataset is valid
        moniker = "EUT{:03d}".format(acq) + "*.hdf5"
        fname = [
            y for x in os.walk(database_path) for y in glob(os.path.join(x[0], moniker))
        ]
        assert fname, "File not found."

        # Load dataset
        f = h5py.File(fname[0], "r")

        # Phantom-specific parameters
        if acq == 1:
            sound_speed = 1603
        elif acq == 2:
            sound_speed = 1618
        elif acq == 3:
            sound_speed = 1607
        elif acq == 4:
            sound_speed = 1614
        elif acq == 5:
            sound_speed = 1495
        else:
            sound_speed = 1479

        # Get data
        self.idata = np.array(f["channel_data"], dtype="float32")
        self.qdata = np.imag(hilbert(self.idata, axis=-1))
        self.angles = np.array(f["transmit_direction"])[:, 0]
        self.fc = np.array(f["modulation_frequency"]).item()
        self.fs = np.array(f["sampling_frequency"]).item()
        self.c = sound_speed
        self.time_zero = np.array(f["start_time"], dtype="float32")[0]
        self.fdemod = 0
        self.ele_pos = np.array(f["element_positions"], dtype="float32").T
        self.ele_pos[:, 0] -= np.mean(self.ele_pos[:, 0])

        # For this dataset, time zero is the center point
        for i, a in enumerate(self.angles):
            self.time_zero[i] = self.ele_pos[-1, 0] * np.abs(np.sin(a)) / self.c

        # Seems to be some offset
        self.time_zero += 10 / self.fc

        # Validate that all information is properly included
        super().validate()
        
class JHUData(PlaneWaveData):
    """ Load data from University of Oslo. """

    def __init__(self, database_path, acq):

        # Make sure the selected dataset is valid
        moniker = "JHU{:03d}".format(acq) + ".hdf5"
        fname = [
            y for x in os.walk(database_path) for y in glob(os.path.join(x[0], moniker))
        ]
        assert fname, "File not found."
        assert acq in list(range(24, 35)), "Focused Data. Use FTDataLoaders"

        # Load dataset
        f = h5py.File(fname[0], "r")

        # Get data
        self.idata = np.array(f["channel_data"], dtype="float32")
        self.qdata = np.imag(hilbert(self.idata, axis=-1))
        self.angles = np.array(f["angles"])
        self.fc = np.array(f["modulation_frequency"]).item()
        self.fs = np.array(f["sampling_frequency"]).item()
        self.c = np.array(f["sound_speed"]).item()
        self.time_zero = -1 * np.array(f["time_zero"], dtype="float32")
        self.fdemod = 0

        xpos = np.array(f["element_positions"], dtype="float32").T
        self.ele_pos = np.stack([xpos, 0 * xpos, 0 * xpos], axis=1)
        self.zlims = np.array([0e-3, self.idata.shape[2] * self.c / self.fs / 2])
        self.xlims = np.array([self.ele_pos[0, 0], self.ele_pos[-1, 0]])

        # For this dataset, time zero is the center point
        # self.time_zero = np.zeros((len(self.angles),), dtype="float32")
        # for i, a in enumerate(self.angles):
        #     self.time_zero[i] = self.ele_pos[-1, 0] * np.abs(np.sin(a)) / self.c

        # Seems to be some offset
        # self.time_zero -= 10 / self.fc

        # Validate that all information is properly included
        super().validate()

class TSHData(PlaneWaveData):
    """ Load data from Tsinghua University. """

    def __init__(self, database_path, acq):
        # Make sure the selected dataset is valid
        moniker = "TSH{:03d}".format(acq) + "*.hdf5"
        fname = [
            y for x in os.walk(database_path) for y in glob(os.path.join(x[0], moniker))
        ]
        assert fname, "File not found."

        # Load dataset
        f = h5py.File(fname[0], "r")

        # Get data
        self.angles = np.array(f["angles"])
        self.idata = np.array(f["channel_data"], dtype="float32")
        self.idata = np.reshape(self.idata, (128, len(self.angles), -1))
        self.idata = np.transpose(self.idata, (1, 0, 2))
        self.qdata = np.imag(hilbert(self.idata, axis=-1))
        self.fc = np.array(f["modulation_frequency"]).item()
        self.fs = np.array(f["sampling_frequency"]).item()
        self.c = 1540  # np.array(f["sound_speed"]).item()
        self.time_zero = np.zeros((len(self.angles),), dtype="float32")
        self.fdemod = 0

        # Make the element positions based on L11-4v geometry
        pitch = 0.3e-3
        nelems = self.idata.shape[1]
        xpos = np.arange(nelems) * pitch
        xpos -= np.mean(xpos)
        self.ele_pos = np.stack([xpos, 0 * xpos, 0 * xpos], axis=1)

        # For this dataset, time zero is the center point
        for i, a in enumerate(self.angles):
            self.time_zero[i] = self.ele_pos[-1, 0] * np.abs(np.sin(a)) / self.c

        # Validate that all information is properly included
        super().validate()

In [3]:
def get_name_and_acq(filename):
    file_text = os.path.splitext(filename)
    name = file_text[0]
    number = int(name[-3:])
    origin = name[0:3]
    return origin, number

## Read all data

In [ ]:
path1 = r'D:\OneDrive\Documents\Maestria_Biomedica\Imagenes_Medicas\model_cubdl\TSH'
files = os.listdir(path1)

max_rows = 0
max_columns = 0 

for file in files:
    if file.endswith(".hdf5"):
        print(file)
        origin, acq = get_name_and_acq(file)
        # print(os.path.abspath(file))
        full_path = os.path.abspath(file)
        if origin== 'MYO':
            print(file)
            P = MYOData(path1, acq)
            xlims = [P.ele_pos[0, 0], P.ele_pos[-1, 0]]
            zlims = [5e-3, 55e-3]
        elif origin == 'INS':
            P = INSData(path1, acq)
            xlims = [P.ele_pos[0, 0], P.ele_pos[-1, 0]]
            zlims = [10e-3, 60e-3]
            if acq >= 13:
                zlims = [10e-3, 50e-3]
        elif origin == 'UFL':
            P = UFLData(path1, acq)
            xlims = [P.ele_pos[0, 0], P.ele_pos[-1, 0]]
            zlims = [10e-3, 50e-3]
        elif origin == 'OSL':
            P = OSLData(path1, acq)
            xlims = [P.ele_pos[0, 0], P.ele_pos[-1, 0]]
            zlims = [10e-3, 65e-3]
            if acq == 10:
                zlims = [5e-3, 50e-3]
        elif origin == 'EUT':
            P = EUTData(path1, acq)
            xlims = [P.ele_pos[0, 0], P.ele_pos[-1, 0]]
            zlims = [10e-3, 80e-3]
        elif origin == 'JHU':
            P = JHUData(path1, acq)
            xlims = [P.ele_pos[0, 0], P.ele_pos[-1, 0]]
            zlims = [0e-3, 30e-3]
        elif origin == 'TSH':
            P = TSHData(path1, acq)
            xlims = [P.ele_pos[0, 0], P.ele_pos[-1, 0]]
            zlims = [10e-3, 45e-3]
        
        wvln = P.c / P.fc
        dx = wvln / 2.5
        dz = dx  # Use square pixels
        grid = make_pixel_grid(xlims, zlims, dx, dz)
        fnum = 1

        x = (P.idata, P.qdata)
        idx = len(P.angles) // 2  # Choose center angle
        das1 = DAS_PW(P, grid, idx, rxfnum=fnum)
        idas1, qdas1 = das1(x)
        idas1, qdas1 = idas1.detach().cpu().numpy(), qdas1.detach().cpu().numpy()
 

# Model training

In [4]:
import os
import sys
import torch
import argparse
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import numpy as np
import PIL
from PIL import Image
import time
import logging
import argparse
from efficientnet_lite import efficientnet_lite_params, build_efficientnet_lite
from utils.train_utils import accuracy, AvgrageMeter, CrossEntropyLabelSmooth, save_checkpoint, get_lastest_model, get_parameters
from torchsummary import summary

## Model definition

In [5]:
model_name = "efficientnet_lite0"
num_outputs = 275544
model = build_efficientnet_lite(model_name, num_outputs)
device = torch.device('cpu')
model = model.to(device)
summary(model,input_size=(2,356,387))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 178, 194]             576
       BatchNorm2d-2         [-1, 32, 178, 194]              64
             ReLU6-3         [-1, 32, 178, 194]               0
            Conv2d-4         [-1, 32, 178, 194]             288
       BatchNorm2d-5         [-1, 32, 178, 194]              64
             ReLU6-6         [-1, 32, 178, 194]               0
            Conv2d-7         [-1, 16, 178, 194]             512
       BatchNorm2d-8         [-1, 16, 178, 194]              32
       MBConvBlock-9         [-1, 16, 178, 194]               0
           Conv2d-10         [-1, 96, 178, 194]           1,536
      BatchNorm2d-11         [-1, 96, 178, 194]             192
            ReLU6-12         [-1, 96, 178, 194]               0
           Conv2d-13           [-1, 96, 89, 97]             864
      BatchNorm2d-14           [-1, 96,

## Pass 1 image

In [6]:
path1 = r'D:\OneDrive\Documents\Maestria_Biomedica\Imagenes_Medicas\model_cubdl\test_image'
files = os.listdir(path1)

for file in files:
    if file.endswith(".hdf5"):

        origin, acq = get_name_and_acq(file)
        # print(os.path.abspath(file))
        full_path = os.path.abspath(file)
        if origin == 'TSH':
            P = TSHData(path1, acq)
            xlims = [P.ele_pos[0, 0], P.ele_pos[-1, 0]]
            zlims = [10e-3, 45e-3]

        wvln = P.c / P.fc
        dx = wvln / 2.5
        dz = dx  # Use square pixels
        grid = make_pixel_grid(xlims, zlims, dx, dz)
        fnum = 1

        # make data from 1 angle
        x = (P.idata, P.qdata)
        idx = len(P.angles) // 2  # Choose center angle
        das1 = DAS_PW(P, grid, idx, rxfnum=fnum)
        idas1, qdas1 = das1(x)
        idas1, qdas1 = idas1.detach().cpu().numpy(), qdas1.detach().cpu().numpy()

        input_data = np.stack((idas1, qdas1), axis=0)
        print(input_data.shape)

        # make data from all angles
        dasN = DAS_PW(P, grid, rxfnum=fnum)
        idasN, qdasN = dasN(x)
        print("idasN.shape: " + str(idasN.shape))
        print("qdasN.shape: " + str(qdasN.shape))
        idasN, qdasN = idasN.detach().cpu().numpy(), qdasN.detach().cpu().numpy()
        target = np.stack((idasN, qdasN), axis=0)
        print(target.shape)

        if (input_data.shape == target.shape):
            print("!!! iguales !!!")
 

(2, 356, 387)
idasN.shape: torch.Size([356, 387])
qdasN.shape: torch.Size([356, 387])
(2, 356, 387)
!!! iguales !!!


In [7]:
import torch.optim as optim

criterion = nn.L1Loss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [8]:
a = np.array([[1, 2, 3],[4, 5, 6],[7, 8, 9]])
b = np.array([[10, 20, 30],[40, 50, 60],[70, 80, 90]])
c = np.stack((a, b), axis=0)
print(c[1])

[[10 20 30]
 [40 50 60]
 [70 80 90]]


In [9]:
input_torch = torch.from_numpy(input_data)
out = model(input_torch[None,...])

In [10]:
out.shape

torch.Size([1, 275544])

In [11]:
out_reshape = torch.reshape(out, (2,356,387))
out_reshape.shape

torch.Size([2, 356, 387])

tensor([[10, 11, 12],
        [13, 14, 15],
        [16, 17, 18]])